TODO:

MODEL
- Results should be deleted between runs
- Make InvestmentDistribution, etc. fields more understandable
- Still need an AssetInput class
- Make some default inputs, like stocks and bonds. Maybe InvestmentDistributionInputs are initialized with a default distribution.
- Make all input labels readable
- Add instructions
- Let Income, Expense, and maybe other classes grow over time (i.e., have an AAGR)

- Maybe: Objects that reference other objects (e.g., transfer_from, investment_distributions, etc) should reference a key to look up those objects, rather than the actual object.
- Maybe allow for randomness besides just market returns, e.g., income amounts, expense amounts?
- Improve results display...
- Incorporate required 401k/IRA withdrawals
- Come up with a way to use machine learning to adjust parameters to minimize the likelihood of ending up with below $1 million, given a certain level of income and spending.




In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label
from IPython.display import display, clear_output, Markdown

In [ ]:
# Got these based from doing some tricky stuff on https://www.ssa.gov/benefits/retirement/planner/AnypiaApplet.html
# Assumes I start withdrawing at 68.
# I think this is just for one person.
social_security_income_by_retirement_age = [
  [range(40, 45), 26808.0],
  [range(45, 50), 30840.0],
  [range(50, 55), 35088.0],
  [range(55, 60), 38448.0],
  [range(60, 68), 41316.0],
  [range(68, 100), 46308.0]
]

In [ ]:
from typing import List

class TaxCalculator:
  def capital_gains_tax_rate(self):
    return 0.15

  def calculate_ny_income_tax(self, income: float) -> float:
    ny_tax_brackets = [
        [0.0, 0.0],
        [0.04, 17150.0],
        [0.045, 23600.0],
        [0.0525, 27900.0],
        [0.055, 161550.0],
        [0.06, 323200.0],
        [0.0685, 2155350.0],
        [0.0965, 5000000.0],
        [0.103, 25000000.0],
        [0.109, float('inf')]

    ]
    ny_standard_deduction = 16050.0
    return self.calculate_tax(income, ny_standard_deduction, ny_tax_brackets)

  def calculate_payroll_tax(self, income: float) -> float:
    payroll_tax_brackets = [
        [0.0765, 0.0],
        [0.0145, 160000.0],
        [0.0235, float('inf')]
    ]
    payroll_standard_deduction = 0.0
    return self.calculate_tax(income, payroll_standard_deduction, payroll_tax_brackets)

  def calculate_federal_income_tax(self, income: float) -> float:
    federal_tax_brackets = [
      [0.0, 0.0],
      [0.1,	22000.0],
      [0.12,	89450.0],
      [0.22,	190750.0],
      [0.24,	364200.0],
      [0.32,	462500.0],
      [0.35,	693650.0],
      [0.37,	float('inf')]
    ]
    federal_standard_deduction = 29200.0
    return self.calculate_tax(income, federal_standard_deduction, federal_tax_brackets)

  def calculate_nyc_income_tax(self, income: float) -> float:
    nyc_tax_brackets = [
        [0.0, 0.0],
        [0.03078, 21600.0],
        [0.03762, 45000.0],
        [0.03819, 90000.0],
        [0.03876, float('inf')]
    ]
    nyc_standard_deduction = 16050.0
    return self.calculate_tax(income, nyc_standard_deduction, nyc_tax_brackets)

  def calculate_tax(self, income: str, deduction: str, tax_brackets: List[List[float]]) -> float:
    taxable_income = income - deduction
    tax_owed = 0
    for bracket in tax_brackets:
      rate, threshold = bracket
      if taxable_income <= 0:
        break
      elif taxable_income <= threshold:
        tax_owed += taxable_income * rate
        break
      else:
        tax_owed += threshold * rate
        taxable_income -= threshold
    return tax_owed


In [ ]:
from typing import List
import random
import math

class InvestmentVehicle:
  def __init__(self, name: str, aagr: float, dynamic_mean: float=0.0, dynamic_std_dev: float=0.0):
    self.name = name
    self.aagr = aagr
    self.dynamic_mean = dynamic_mean
    self.dynamic_std_dev = dynamic_std_dev

  def conditionally_reset_aagr(self, dynamic):
    if dynamic and not math.isclose(self.dynamic_mean, 0):
      self.aagr = self.dynamic_mean + self.dynamic_std_dev * random.normalvariate(0, 1)

class InvestmentProportion:
  def __init__(self, investment_vehicle: InvestmentVehicle, proportion: float):
    self.investment_vehicle = investment_vehicle
    self.proportion = proportion

class InvestmentDistribution:
  def __init__(self, years: List[int], investment_proportions: List[InvestmentProportion]):
    self.years = years
    self.investment_proportions = investment_proportions

    if sum(investment_proportion.proportion for investment_proportion in investment_proportions) != 1:
      raise ValueError("Investment proportions must sum to 1")

  def annual_growth(self, balance: float) -> float:
    growth = 0
    for investment_proportion in self.investment_proportions:
      growth += balance * investment_proportion.proportion * investment_proportion.investment_vehicle.aagr
    return growth

In [ ]:
class ACCOUNT_TYPE:
  BANK= "bank"
  RETIREMENT= "retirement"
  INVESTMENT= "investments"
  FIVE_TWO_NINE= "529"
  # Note: The order of this is the order in which they should be withdrawn from to pay for things
  ALL = [FIVE_TWO_NINE, BANK, INVESTMENT, RETIREMENT]

In [ ]:
class WITHDRAWAL_TAX_TYPE:
  INCOME="income"
  CAPITAL_GAINS="capital gains"
  NONE="none"
  ALL=[INCOME, CAPITAL_GAINS, NONE]

class Withdrawal:
  def __init__(self, amount: float, account_type: ACCOUNT_TYPE, capital_gains: float=0.0):
    self.amount = amount
    self.account_type = account_type
    self.capital_gains = capital_gains

  def tax_type(self):
    if self.account_type == ACCOUNT_TYPE.INVESTMENT:
      return WITHDRAWAL_TAX_TYPE.CAPITAL_GAINS
    elif self.account_type == ACCOUNT_TYPE.RETIREMENT:
      return WITHDRAWAL_TAX_TYPE.INCOME
    else:
      return WITHDRAWAL_TAX_TYPE.NONE

In [ ]:
from typing import List, Tuple, Optional
import math

class Account:
  def __init__(self, name: str, account_type: ACCOUNT_TYPE, starting_balance: float, investment_distributions: List[InvestmentDistribution], earliest_withdrawal_year: int=0):
    self.name = name
    self.account_type = account_type
    self.investment_distributions = investment_distributions
    self.earliest_withdrawal_year = earliest_withdrawal_year
    self.principal = starting_balance
    self.gains = 0.0

  def deposit(self, amount: float):
    self.principal += amount

  def apply_annual_growth(self, year: int):
    growth = 0

    for investment_distribution in self.investment_distributions:
      if year in investment_distribution.years:
        growth += investment_distribution.annual_growth(self.balance())
        break

    self.gains += growth

  def balance(self):
    return self.principal + self.gains

  def withdraw(self, amount: float) -> Withdrawal:
    if math.isclose(amount, 0):
      return Withdrawal(0.0, self.account_type)

    if self.balance() < amount:
      raise ValueError("Insufficient funds")

    proportion_of_balance_that_is_gains = self.gains / self.balance()
    gains_reduction = amount * proportion_of_balance_that_is_gains
    principal_reduction = amount * (1 - proportion_of_balance_that_is_gains)
    self.gains -= gains_reduction
    self.principal -= principal_reduction

    return Withdrawal(amount, self.account_type, capital_gains=gains_reduction)



In [ ]:
class Expense:
  def __init__(self, name: str, amount: float, year: int, five_two_nine_eligible: bool=False):
    self.name = name
    self.amount = amount
    self.year = year
    self.five_two_nine_eligible = five_two_nine_eligible

  def pay(self, payment_amount: float):
    self.amount -= payment_amount

In [ ]:
class Income:
  def __init__(self, name: str, amount: float, year: int, deposit_in: 'Account', federal_income_tax: bool=True, payroll_tax: bool=True, ny_income_tax: bool=True, nyc_income_tax: bool=True):
    self.name = name
    self.amount = amount
    self.year = year
    self.federal_income_tax = federal_income_tax
    self.payroll_tax = payroll_tax
    self.ny_income_tax = ny_income_tax
    self.nyc_income_tax = nyc_income_tax
    self.deposit_in = deposit_in

In [ ]:
class Gift:
  def __init__(self, name: str, amount: float, year: int, account: 'Account'):
    self.name = name
    self.amount = amount
    self.year = year
    self.account = account

  def receive(self):
    self.account.deposit(self.amount)

In [ ]:
class Debt:
  def __init__(self, name: str, amount: float, aagr: float, scheduled: bool=False, five_two_nine_eligible: bool=False):
    self.name = name
    self.amount = amount
    self.aagr = aagr
    self.scheduled = scheduled
    self.five_two_nine_eligible = five_two_nine_eligible

  def pay(self, payment_amount: float):
    self.amount -= payment_amount

  def add(self, amount: float) -> float:
    self.amount += amount

  def apply_annual_growth(self):
    self.amount *= (1 + self.aagr)

In [ ]:
from typing import Tuple, Optional, List

class Transfer:
  def __init__(self, name: str, amount: float, year: int, transfer_from: Account, transfer_to: Account | Debt, required: bool=False):
    self.name = name
    self.amount = amount
    self.year = year
    self.transfer_from = transfer_from
    self.transfer_to = transfer_to
    self.required = required

  # If there is not enough in the transfer_from account and the full amount is required, returns an expense to
  def execute(self) -> Tuple[Withdrawal, Optional[Expense]]:
    expense = None
    withdrawal = None

    if self.transfer_from.balance() < self.amount and self.required:
      withdrawal_amount = self.transfer_from.balance()
      expense = Expense(f"insufficient funds for {self.name}", self.amount - withdrawal_amount, self.year)
      withdrawal = self.transfer_from.withdraw(withdrawal_amount)
      self.execute_transfer_to(self.amount)
    else:
      withdrawal_amount = min(self.transfer_from.balance(), self.amount)
      withdrawal = self.transfer_from.withdraw(withdrawal_amount)
      self.execute_transfer_to(withdrawal_amount)

    return withdrawal, expense

  def execute_transfer_to(self, amount):
    if isinstance(self.transfer_to, Account):
      self.transfer_to.deposit(amount)
    elif isinstance(self.transfer_to, Debt):
      self.transfer_to.pay(amount)
    else:
      raise TypeError("Variable must be an instance of Account or Debt")

In [ ]:
class Asset:
  def __init__(self, name: str, value: float, aagr: float):
    self.name = name
    self.value = value
    self.aagr = aagr

  def apply_annual_growth(self):
    self.value += self.value * self.aagr

In [ ]:
from typing import List

class Payer:
  ## Returns an array of withdrawals and a dollar amount for any outstanding debt
  @classmethod
  def attempt_to_pay_payables(cls, year: int, payables: List[Expense | Debt], accounts: List[Account]) -> List[Withdrawal]:
    withdrawals = []

    # sort payables with five_two_nine_eligible payables first
    payables.sort(key=lambda p: p.five_two_nine_eligible, reverse=True)
    # sort accounts according to the order of their account type in ACCOUNT_TYPE.ALL
    accounts.sort(key=lambda a: ACCOUNT_TYPE.ALL.index(a.account_type))

    for payable in payables:
      for account in accounts:
        if year < account.earliest_withdrawal_year:
          continue
        if account.account_type == ACCOUNT_TYPE.FIVE_TWO_NINE and not payable.five_two_nine_eligible:
          continue
        while account.balance() > 0 and payable.amount > 0:
          withdrawal_amount = min(
            account.balance(),
            payable.amount
          )

          withdrawal = account.withdraw(withdrawal_amount)
          withdrawals.append(withdrawal)
          payable.pay(withdrawal.amount)

    return withdrawals

In [ ]:
class Aggregator:
  def __init__(self):
    self.net_worth = []
    self.assets = []
    self.income = []
    self.retirement = []
    self.investment = []
    self.five_two_nine = []
    self.bank_account = []
    self.debt = []

  def append(self, another_aggregator: 'Aggregator') -> 'Aggregator':
    self.net_worth.append(another_aggregator.net_worth)
    self.income.append(another_aggregator.income)
    self.retirement.append(another_aggregator.retirement)
    self.investment.append(another_aggregator.investment)
    self.five_two_nine.append(another_aggregator.five_two_nine)
    self.bank_account.append(another_aggregator.bank_account)
    self.debt.append(another_aggregator.debt)
    self.assets.append(another_aggregator.assets)
    return self

In [ ]:
from typing import List
from typing import Tuple

class HousePurchase:
  def __init__(self, home_price: int, annual_interest_rate: float, year: int, last_year: int, down_payment_acct_src: Account, mortgage_acct_src: Account, down_payment_proportion: float=0.2):
    self.home_price = home_price
    self.annual_interest_rate = annual_interest_rate
    self.year = year
    self.last_year = last_year
    self.down_payment_acct_src = down_payment_acct_src
    self.mortgage_acct_src = mortgage_acct_src
    self.loan_term_years = 30
    self.down_payment_proportion = down_payment_proportion
    self.tax_rate = 0.0071

  # Return values:
  # - Assets
  #   - The value of the house
  # - Expenses:
  #   - Closing costs (once)
  #   - Property taxes (forever, 0.71% of value) (how does tax assessment work?)
  #   - Interest portion of mortgage payment (need to figure out when/how to deduct from federal taxable income) (30 years)
  # - Transfers
  #   - Down payment (transfer_from=[boa, vanguard], transfer_to=debt)
  #   - Annual paydowns of the mortgage principal (transfer_from=[boa, vanguard], transfer_to=debt)
  # - Debts:
  #   - The house price
  def execute(self) -> Tuple[Asset, Debt, List[Expense], List[Transfer]]:
    house = Asset("house", self.home_price, 0.01)
    debt = Debt("mortgage", self.home_price, 0.0, scheduled=True)

    expenses = []
    transfers = []

    # Expenses
    for y in range(self.year, self.last_year):
      expenses.append(Expense("property taxes", self.home_price*self.tax_rate, y))

    # Values based mainly on NYTIMES buy vs rent calculator
    expenses.append(Expense("house closing costs", self.home_price*0.04, self.year))

    for y in range(self.year, self.last_year):
      expenses.append(Expense(f"maintenance, insurance, extra utilities {y}", self.home_price*0.0155+1200, y))

    down_payment = Transfer("down payment", self.home_price*self.down_payment_proportion, self.year, transfer_from=self.down_payment_acct_src, transfer_to=debt, required=True)
    transfers.append(down_payment)

    annual_breakdowns = self.calculate_annual_interest_and_principal(self.home_price - down_payment.amount)
    for i, (interest_paid, principal_paid) in enumerate(annual_breakdowns):
      y = self.year + i
      expenses.append(Expense(f"mortgage interest paid {y}", interest_paid, y))
      transfers.append(Transfer(f"mortgage principal paid {y}", principal_paid, y, transfer_from=self.mortgage_acct_src, transfer_to=debt, required=True))

    return house, debt, expenses, transfers

  # Returns a list of [[interest_paid, principal_paid], [...], ...] for each year.
  def calculate_annual_interest_and_principal(self, loan_amount) -> List[List[float]]:
    monthly_interest_rate = self.annual_interest_rate / 12

    # Total number of monthly payments
    total_payments = self.loan_term_years * 12

    # Monthly mortgage payment
    if monthly_interest_rate > 0:
      monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -total_payments)
    else:
      monthly_payment = loan_amount / total_payments

    # Initialize balance
    balance = loan_amount
    annual_breakdowns = []

    for i in range(0, self.loan_term_years):
      interest_paid = 0
      principal_paid = 0

      for month in range(1, 13):
        interest_payment = balance * monthly_interest_rate
        principal_payment = monthly_payment - interest_payment
        balance -= principal_payment

        interest_paid += interest_payment
        principal_paid += principal_payment

      annual_breakdowns.append([interest_paid, principal_paid])

    return annual_breakdowns

In [ ]:
from typing import Tuple, List
from operator import ge

class YearSimulator:
  @classmethod
  def execute(cls, year: int, investment_vehicles: List[InvestmentVehicle], accounts: List[Account], expenses: List[Expense], incomes: List[Income], transfers: List[Transfer], debts: List[Debt], assets: List[Asset], gifts: List[Gift], debug: bool=False, dynamic: bool=False) -> Tuple[List[Account], List[Expense], List[Income], List[Transfer], List[Debt], List[Asset], List[Gift]]:
    # SETUP
    withdrawals = []
    starting_tax_liability = 0.0
    extra_taxes = 0.0
    tax_calculator = TaxCalculator()
    for iv in investment_vehicles:
      iv.conditionally_reset_aagr(dynamic)


    annual_incomes = [income for income in incomes if income.year == year]
    annual_gifts = [gift for gift in gifts if gift.year == year]
    annual_transfers = [transfer for transfer in transfers if transfer.year == year]

    if debug:
      print(f"*** START OF {year} ***")
      for a in accounts:
        print(f"[{year}] [account] {a.name}: {a.balance()}")
      for d in debts:
        print(f"[{year}] [debt] {d.name}: {d.amount} ({d.scheduled})")
      for t in annual_transfers:
        print(f"[{year}] [transfer] {t.name}: {t.amount}")
      for g in annual_gifts:
        print(f"[{year}] [gift] {g.name}: {g.amount}")
      for i in annual_incomes:
        print(f"[{year}] [income] {i.name}: {i.amount}")
      for e in expenses:
        if e.year == year:
          print(f"[{year}] [expense] {e.name}: {e.amount}")


    # DEPOSIT INCOME
    for income in annual_incomes:
      income.deposit_in.deposit(income.amount)


    # GIFTS
    for gift in annual_gifts:
      gift.receive()


    # TRANSFERS
    for transfer in annual_transfers:
      withdrawal, expense = transfer.execute()
      withdrawals.append(withdrawal)
      if expense is not None:
        expenses.append(expense)


    # CALCULATE TAXES
    payroll_taxes = 0.0
    for income in annual_incomes:
      if income.payroll_tax:
        payroll_taxes += tax_calculator.calculate_payroll_tax(income.amount)

    # TODO: This will overestimate the retirement contribution when there aren't sufficient funds in the transfer_from account
    retirement_transfer = sum(transfer.amount for transfer in annual_transfers if isinstance(transfer.transfer_to, Account) and transfer.transfer_to.account_type == ACCOUNT_TYPE.RETIREMENT)

    federal_tax_eligible_income = -(min(retirement_transfer, 45000))
    for income in annual_incomes:
      if income.federal_income_tax:
        federal_tax_eligible_income += income.amount
    federal_income_taxes = tax_calculator.calculate_federal_income_tax(federal_tax_eligible_income)

    five_two_nine_transfer = sum(transfer.amount for transfer in annual_transfers if isinstance(transfer.transfer_to, Account) and transfer.transfer_to.account_type == ACCOUNT_TYPE.FIVE_TWO_NINE)

    ny_tax_eligible_income = -(min(retirement_transfer, 45000) + min(five_two_nine_transfer, 10000))
    for income in annual_incomes:
      if income.ny_income_tax:
        ny_tax_eligible_income += income.amount
    ny_income_taxes = tax_calculator.calculate_ny_income_tax(ny_tax_eligible_income)

    nyc_tax_eligible_income = -(min(retirement_transfer, 45000) + min(five_two_nine_transfer, 10000))
    for income in annual_incomes:
      if income.nyc_income_tax:
        nyc_tax_eligible_income += income.amount
    nyc_income_taxes = tax_calculator.calculate_nyc_income_tax(nyc_tax_eligible_income)

    expenses.append(Expense(f"taxes for {year}", payroll_taxes + federal_income_taxes + ny_income_taxes + nyc_income_taxes, year))
    annual_expenses = [e for e in expenses if e.year == year]


    #####   PAY EXPENSES  ######

    w = Payer.attempt_to_pay_payables(year, annual_expenses, accounts)
    withdrawals.extend(w)

    unpaid_expenses = sum(expense.amount for expense in annual_expenses)
    # TODO: The aagr of unpaid expenses should live somewhere rather than be hardcoded here
    debts.append(Debt(f"unpaid expenses for {year}", unpaid_expenses, 0.10))


    #####   PAY UNSCHEDULED DEBT  ######
    unscheduled_debts = [debt for debt in debts if debt.amount > 0 and not debt.scheduled]
    w = Payer.attempt_to_pay_payables(year, unscheduled_debts, accounts)
    withdrawals.extend(w)

    if debug:
      for w in withdrawals:
        print(f"[{year}] [withdrawal] {w.amount}, {w.account_type}, (capital gains {w.capital_gains})")

    #####   NEXT YEAR'S TAXES  ######
    extra_taxes = 0.0
    extra_income = 0.0
    capital_gains = 0.0
    for withdrawal in withdrawals:
      if withdrawal.tax_type() == WITHDRAWAL_TAX_TYPE.INCOME:
        extra_income += withdrawal.amount
      elif withdrawal.tax_type() == WITHDRAWAL_TAX_TYPE.CAPITAL_GAINS:
        capital_gains += withdrawal.capital_gains

    extra_taxes += tax_calculator.calculate_federal_income_tax(federal_tax_eligible_income + extra_income) - federal_income_taxes
    extra_taxes += tax_calculator.calculate_ny_income_tax(ny_tax_eligible_income + extra_income) - ny_income_taxes
    extra_taxes += tax_calculator.calculate_nyc_income_tax(nyc_tax_eligible_income + extra_income) - nyc_income_taxes
    extra_taxes += capital_gains * tax_calculator.capital_gains_tax_rate()

    expenses.append(Expense(f"extra taxes for {year}", extra_taxes, year+1))


    if debug:
      print(f"*** END OF {year} ***")
      for a in accounts:
        print(f"[{year}] [account] {a.name}: {a.balance()}")
      for d in debts:
        print(f"[{year}] [debt] {d.name}: {d.amount} ({d.scheduled})")
      for t in annual_transfers:
        print(f"[{year}] [transfer] {t.name}: {t.amount}")
      for g in annual_gifts:
        print(f"[{year}] [gift] {g.name}: {g.amount}")
      for i in annual_incomes:
        print(f"[{year}] [income] {i.name}: {i.amount}")
      for e in expenses:
        if e.year == year:
          print(f"[{year}] [expense] {e.name}: {e.amount}")


    # APPLY ANNUAL GROWTH
    for account in accounts:
      account.apply_annual_growth(year)

    for debt in debts:
      debt.apply_annual_growth()

    for asset in assets:
      asset.apply_annual_growth()

    return accounts, expenses, incomes, transfers, debts, assets, gifts

In [ ]:
class FormDataParser:
  def __init__(self, data: dict):
    self.data = data
    self.investment_vehicles = []
    self.accounts = []
    self.expenses = []
    self.incomes = []
    self.transfers = []
    self.debts = []
    self.assets = []
    self.gifts = []

    for debt_input in self.data["debts"]:
      self.debts.append(Debt(**debt_input))

    for investment_vehicle_input in self.data["investment_vehicles"]:
      self.investment_vehicles.append(InvestmentVehicle(**investment_vehicle_input))

    for account_input in self.data["accounts"]:
      investment_distributions = []
      investment_distributions_input = account_input.pop("investment_distributions", [])

      for investment_distribution_input in investment_distributions_input:
        investment_proportions = []
        investment_proportions_input = investment_distribution_input.pop("investment_proportions", [])

        for investment_proportion_input in investment_proportions_input:
          investment_vehicle = self.find_object_by_name(self.investment_vehicles, investment_proportion_input.pop("investment_vehicle", None))
          investment_proportions.append(InvestmentProportion(**investment_proportion_input, investment_vehicle=investment_vehicle))

        investment_distributions.append(InvestmentDistribution(**investment_distribution_input, investment_proportions=investment_proportions))

      self.accounts.append(Account(**account_input, investment_distributions=investment_distributions))

    for gift_input in self.data["gifts"]:
      account = self.find_object_by_name(self.accounts, gift_input.pop("account", None))
      years = gift_input.pop("years", [])
      for year in years:
        self.gifts.append(Gift(**gift_input, account=account, year=year))

    for expense_input in self.data["expenses"]:
      years = expense_input.pop("years", [])
      for year in years:
        self.expenses.append(Expense(**expense_input, year=year))

    for income_input in self.data["incomes"]:
      deposit_in = self.find_object_by_name(self.accounts, income_input.pop("deposit_in", None))
      years = income_input.pop("years", [])
      for year in years:
        self.incomes.append(Income(**income_input, year=year, deposit_in=deposit_in))

    for transfer_input in self.data["transfers"]:
      transfer_from = self.find_object_by_name(self.accounts, transfer_input.pop("transfer_from", None))
      transfer_to = self.find_object_by_name(self.accounts, transfer_input.pop("transfer_to", None))
      years = transfer_input.pop("years", [])
      for year in years:
        self.transfers.append(Transfer(**transfer_input, year=year, transfer_from=transfer_from, transfer_to=transfer_to))

  def find_object_by_name(self, objects, target_name):
    for obj in objects:
        if obj.name == target_name:
            return obj
    raise ValueError(f"can't find object with name {target_name}")  # Raise if no match is found




In [ ]:
from typing import List, Tuple
import copy

class Simulations:
  def __init__(self, data: dict, first_year: int, last_year: int, dynamic: bool=False, debug: bool=False):
    self.data = data
    self.first_year = first_year
    self.last_year = last_year
    self.dynamic = dynamic
    self.debug = debug

  def execute_simulation(self, years: List[int], investment_vehicles: List[InvestmentVehicle], accounts: List[Account], expenses: List[Expense], incomes: List[Income], transfers: List[Transfer], debts: List[Debt], assets: List[Asset], gifts: List[Gift], debug: bool=False, dynamic: bool=True) -> Aggregator:
    aggregator = Aggregator()
    for year in years:
      accounts, expenses, incomes, transfers, debts, assets, gifts = YearSimulator.execute(year=year, investment_vehicles=investment_vehicles, accounts=accounts, expenses=expenses, incomes=incomes, transfers=transfers, debts=debts, assets=assets, gifts=gifts, debug=debug, dynamic=dynamic)
      aggregator.net_worth.append(sum(acct.balance() for acct in accounts) + sum(asset.value for asset in assets) - sum(d.amount for d in debts))
      aggregator.assets.append(sum(asset.value for asset in assets))
      aggregator.five_two_nine.append(sum(account.balance() for account in accounts if account.account_type == ACCOUNT_TYPE.FIVE_TWO_NINE))
      aggregator.bank_account.append(sum(account.balance() for account in accounts if account.account_type == ACCOUNT_TYPE.BANK))
      aggregator.investment.append(sum(account.balance() for account in accounts if account.account_type == ACCOUNT_TYPE.INVESTMENT))
      aggregator.retirement.append(sum(account.balance() for account in accounts if account.account_type == ACCOUNT_TYPE.RETIREMENT))
      aggregator.debt.append(sum(d.amount for d in debts))
    return aggregator

  def execute(self) -> Aggregator:
    aggregator = Aggregator()
    years = list(range(self.first_year, self.last_year))

    number_of_simulations = 1
    if self.dynamic:
      number_of_simulations = 1000

    for i in range(0, number_of_simulations):
      parsed = FormDataParser(copy.deepcopy(self.data))
      results = self.execute_simulation(years, investment_vehicles=parsed.investment_vehicles, accounts=parsed.accounts, expenses=parsed.expenses, incomes=parsed.incomes, transfers=parsed.transfers, debts=parsed.debts, assets=parsed.assets, gifts=parsed.gifts, debug=debug, dynamic=dynamic)
      aggregator.append(results)
    return aggregator


In [ ]:
from typing import List
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

class ResultsDisplayer:
  def __init__(self):
    # Output widget to hold the plot
    self.plot_output = widgets.Output()

  def display(self, first_year: int, last_year: int, aggregator: Aggregator, dynamic: bool=False, debug: bool=False):
    with self.plot_output:
      # Clear the plot output
      self.plot_output.clear_output(wait=True)
      years = list(range(first_year, last_year))
      df = pd.DataFrame(aggregator.net_worth, columns=years)
      number_of_simulations = df.shape[0]

      # All net worths
      plt.figure(figsize=(10, 6))

      # Net worth median
      median_trajectory = df.median(axis=0)
      plt.plot(years, median_trajectory, linewidth=2, label='Median')

      # Up to 50 trajctories
      num_trajectories_to_plot = min(50, df.shape[0])
      for i in range(num_trajectories_to_plot):
          plt.plot(years, df.iloc[i], linewidth=1, color="gray", alpha=0.5)  # alpha controls transparency

      if debug:
        # Components
        retirement_df  = pd.DataFrame(aggregator.retirement, columns=years)
        plt.plot(years, retirement_df.median(axis=0), color='blue', linewidth=1, label='Retirement')

        investments_df  = pd.DataFrame(aggregator.investment, columns=years)
        plt.plot(years, investments_df.median(axis=0), color='green', linewidth=1, label='Investments')

        five_two_nine_df  = pd.DataFrame(aggregator.five_two_nine, columns=years)
        plt.plot(years, five_two_nine_df.median(axis=0), color='brown', linewidth=1, label='529')

        debt_df  = pd.DataFrame(aggregator.debt, columns=years)
        plt.plot(years, -debt_df.median(axis=0), color='purple', linewidth=1, label='Debt')

        bank_account_df  = pd.DataFrame(aggregator.bank_account, columns=years)
        plt.plot(years, bank_account_df.median(axis=0), color='yellow', linewidth=1, label='Bank account')

        assets_df  = pd.DataFrame(aggregator.assets, columns=years)
        plt.plot(years, assets_df.median(axis=0), color='orange', linewidth=1, label='Assets')

      if dynamic:
        # Calculate percentages
        below_one_million_count = (df[last_year-1] < 1000000).sum()
        below_zero_count = (df[last_year-1] < 0).sum()

        below_one_million_percent = (below_one_million_count / number_of_simulations) * 100
        below_zero_percent = (below_zero_count / number_of_simulations) * 100

        # Display results
        median_net_worth = "${:,.0f}".format(median_trajectory.iloc[-1])
        result_string = f"## Net Worth in {last_year}\n\n###Median: {median_net_worth}###\n\n###Likelihood under $1M: {below_one_million_percent:.0f}%###\n\n###Likelihood under $0: {below_zero_percent:.0f}%###"
        display(Markdown(result_string))

      # Customize the plot
      plt.xlabel('Age')
      plt.ylabel('Net Worth')
      plt.legend()

      # Format y-axis labels in millions
      plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,.1f}M".format(x/1e6)))

      # If dynamic, scale y-axis according to the 33th/67th percentile
      if dynamic:
        plt.ylim(df.quantile(0.33, axis=0).min(), df.quantile(0.67, axis=0).max())

      # Max x-axis dark
      plt.axhline(y=0, color='black', linewidth=1.5)

      plt.grid(True)
      plt.show()

    display(self.plot_output)



In [ ]:
#   REAL UNDERLYING DATA

#   stocks = InvestmentVehicle(name="s&p", aagr=0.0655, dynamic_mean=0.077, dynamic_std_dev=0.175)
#   bonds = InvestmentVehicle(name="bond", aagr=0.0352, dynamic_mean=0.039, dynamic_std_dev=0.0855)

#   fidelity_investment_distributions = [
#     InvestmentDistribution(list(range(2024, 2031)), investment_proportions=[InvestmentProportion(stocks, 0.9), InvestmentProportion(bonds, 0.1)]),
#     InvestmentDistribution(list(range(2031, 2037)), investment_proportions=[InvestmentProportion(stocks, 0.8), InvestmentProportion(bonds, 0.2)]),
#     InvestmentDistribution(list(range(2037, 2043)), investment_proportions=[InvestmentProportion(stocks, 0.7), InvestmentProportion(bonds, 0.3)]),
#     InvestmentDistribution(list(range(2043, 2049)), investment_proportions=[InvestmentProportion(stocks, 0.6), InvestmentProportion(bonds, 0.4)]),
#     InvestmentDistribution(list(range(2049, last_year)), investment_proportions=[InvestmentProportion(stocks, 0.5), InvestmentProportion(bonds, 0.5)])
#   ]

#   nysaves_investment_distributions = [
#     InvestmentDistribution(list(range(2024, 2027)), investment_proportions=[InvestmentProportion(stocks, 0.9), InvestmentProportion(bonds, 0.1)]),
#     InvestmentDistribution(list(range(2027, 2030)), investment_proportions=[InvestmentProportion(stocks, 0.8), InvestmentProportion(bonds, 0.2)]),
#     InvestmentDistribution(list(range(2030, 2033)), investment_proportions=[InvestmentProportion(stocks, 0.7), InvestmentProportion(bonds, 0.3)]),
#     InvestmentDistribution(list(range(2033, 2037)), investment_proportions=[InvestmentProportion(stocks, 0.6), InvestmentProportion(bonds, 0.4)]),
#     InvestmentDistribution(list(range(2037, last_year)), investment_proportions=[InvestmentProportion(stocks, 0.5), InvestmentProportion(bonds, 0.5)])
#   ]

#   vanguard_investment_distributions = [
#     InvestmentDistribution(list(range(2024, 2049)), investment_proportions=[InvestmentProportion(stocks, 0.9), InvestmentProportion(bonds, 0.1)]),
#     InvestmentDistribution(list(range(2049, last_year)), investment_proportions=[InvestmentProportion(stocks, 0.5), InvestmentProportion(bonds, 0.5)])
#   ]

#   boa_investment_distributions = []

#   boa = Account("bank of america", ACCOUNT_TYPE.BANK, 30000.0, boa_investment_distributions)
#   vanguard = Account("vanguard", ACCOUNT_TYPE.INVESTMENT, 150000.0, vanguard_investment_distributions)
#   fidelity = Account("fidelity", ACCOUNT_TYPE.RETIREMENT, 730000.0, fidelity_investment_distributions, earliest_withdrawal_year=2039)
#   nysaves = Account("nysaves", ACCOUNT_TYPE.FIVE_TWO_NINE, 85000.0, nysaves_investment_distributions)
#   accounts = [boa, vanguard, fidelity, nysaves]

#   expenses = []
#   for y in [2027, 2037, 2047, 2057, 2067]:
#     expenses.append(Expense("new car", 40000, y))

#   for y in [2024, 2025, 2026]:
#     expenses.append(Expense("zach daycare", 15000, y))

#   for y in list(range(2024, 2044)):
#     expenses.append(Expense("copake payment", 12000, y))

#   for y in [2037, 2038, 2039, 2040]:
#     expenses.append(Expense("sloane college", 50000, y, five_two_nine_eligible=True))

#   for y in [2040, 2041, 2042, 2043]:
#     expenses.append(Expense("zach college", 50000, y, five_two_nine_eligible=True))

#   for y in range(2024, last_year):
#     expenses.append(Expense("every day spending", 101500, y))


#   incomes = []

#   for y in range(2052, last_year):
#     incomes.append(Income("david social security", 35000, y, deposit_in=boa, payroll_tax=False))

#   for y in range(2024, 2039):
#     incomes.append(Income("david full-time work", 130000, y, deposit_in=boa))

#   for y in range(2024, 2039):
#     incomes.append(Income("claire full-time work", 120000, y, deposit_in=boa))

#   for y in range(2039, 2049):
#     incomes.append(Income("david part-time work", 70000, y, deposit_in=boa))

#   for y in range(2039, 2049):
#     incomes.append(Income("claire part-time work", 80000, y, deposit_in=boa))

#   for y in range(2052, last_year):
#     incomes.append(Income("claire social security", 35000, y, deposit_in=boa, payroll_tax=False))

#   transfers = []

#   gifts = []
#   for y in range(2024, 2037):
#     gifts.append(Gift("dad 529 contribution", 8000, y, nysaves))

#   assets = []

#   debts = []

#   ### RENTAL SCENARIO ###
#   for y in range(2024, last_year):
#     # expenses.append(Expense("rent", 31500, y)) # Current rent
#     expenses.append(Expense("rent", 48000, y)) # More rent
#   for y in range(2024, 2039):
#     transfers.append(Transfer(ACCOUNT_TYPE.RETIREMENT, 45000, y, transfer_from=boa, transfer_to=fidelity))
#   for y in range(2024, 2037):
#     transfers.append(Transfer(ACCOUNT_TYPE.FIVE_TWO_NINE, 1200, y, transfer_from=boa, transfer_to=nysaves))


#   ### HOME PURCHASE SCENARIO ###
#   # house_asset, house_debt, house_expenses, house_transfers = HousePurchase(800000, 0.07, 2024, last_year, down_payment_acct_src=boa, mortgage_acct_src=boa, down_payment_proportion=0.1).execute()
#   # assets.append(house_asset)
#   # debts.append(house_debt)
#   # expenses.extend(house_expenses)
#   # transfers.extend(house_transfers)
#   # gifts.append(Gift("down payment help", 100000, 2024, boa))


# FORM DATA
# data = {'debts': [{'name': 'some random debt', 'amount': 1000.0, 'aagr': 0.03}], 'accounts': [{'name': 'vanguard', 'account_type': 'investments', 'starting_balance': 150000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': [{'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}]}, {'name': 'fidelity', 'account_type': 'retirement', 'starting_balance': 730000.0, 'earliest_withdrawal_year': 2039, 'investment_distributions': [{'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2031, 2032, 2033, 2034, 2035, 2036], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.8}, {'investment_vehicle': 'bonds', 'proportion': 0.2}]}, {'years': [2037, 2038, 2039, 2040, 2041, 2042], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.7}, {'investment_vehicle': 'bonds', 'proportion': 0.3}]}, {'years': [2043, 2044, 2045, 2046, 2047, 2048], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.6}, {'investment_vehicle': 'bonds', 'proportion': 0.4}]}, {'years': [2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.5}, {'investment_vehicle': 'bonds', 'proportion': 0.5}]}]}, {'name': 'boa', 'account_type': 'bank', 'starting_balance': 30000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': []}, {'name': 'nysaves', 'account_type': '529', 'starting_balance': 85000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': [{'years': [2024, 2025, 2026], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2027, 2028, 2029], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.8}, {'investment_vehicle': 'bonds', 'proportion': 0.2}]}, {'years': [2030, 2031, 2032], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.7}, {'investment_vehicle': 'bonds', 'proportion': 0.3}]}, {'years': [2033, 2034, 2035, 2036], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.6}, {'investment_vehicle': 'bonds', 'proportion': 0.4}]}, {'years': [2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.5}, {'investment_vehicle': 'bonds', 'proportion': 0.5}]}]}], 'incomes': [{'name': 'david full-time', 'amount': 130000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'claire full-time', 'amount': 120000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'david part-time', 'amount': 70000.0, 'years': [2040, 2041, 2042, 2043, 2044], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'claire part-time', 'amount': 80000.0, 'years': [2040, 2041, 2042, 2043, 2044], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}], 'expenses': [{'name': 'car', 'amount': 40000.0, 'years': [2027, 2037, 2047, 2057, 2067], 'five_two_nine_eligible': False}, {'name': 'zach college', 'amount': 50000.0, 'years': [2039, 2040, 2041, 2042], 'five_two_nine_eligible': True}, {'name': 'sloane college', 'amount': 50000.0, 'years': [2037, 2038, 2039, 2040], 'five_two_nine_eligible': True}, {'name': 'copake', 'amount': 12000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044], 'five_two_nine_eligible': False}, {'name': 'every day expenses', 'amount': 101500.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'five_two_nine_eligible': False}, {'name': 'zach daycare', 'amount': 20000.0, 'years': [2024, 2025, 2026], 'five_two_nine_eligible': False}], 'gifts': [{'name': 'dad 529', 'amount': 8000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037], 'account': 'nysaves'}], 'transfers': [{'name': 'retirement contribution', 'amount': 45000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'transfer_from': 'boa', 'transfer_to': 'fidelity', 'required': False}], 'investment_vehicles': [{'name': 'stocks', 'aagr': 0.0655, 'dynamic_mean': 0.077, 'dynamic_std_dev': 0.1175}, {'name': 'bonds', 'aagr': 0.0352, 'dynamic_mean': 0.039, 'dynamic_std_dev': 0.0855}]}


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def parse_years(input_string):
    years = set()
    input_string = input_string.replace(" ", "")  # Remove any spaces
    parts = input_string.split(',')

    for part in parts:
        if '-' in part:
            try:
                start, end = part.split('-')
                start, end = int(start), int(end)
                if start > end:
                    start, end = end, start
                years.update(range(start, end + 1))
            except ValueError:
                continue  # Skip invalid ranges
        else:
            try:
                year = int(part)
                years.add(year)
            except ValueError:
                continue  # Skip invalid years

    return sorted(years)


In [ ]:
input_layout = widgets.Layout(border='solid 0.5px gray', padding='10px', margin='10px', width='auto')
inputs_layout = widgets.Layout(border='solid 2px black', padding='0 10px 10px 10px', margin='0 10px 10px 10px', width='auto')

none = widgets.HTML(value="<p>None</p>")

def inputsGroup(h_size, title, input_widgets, add_btn):
    inputs = [widgets.HTML(value="<p>None</p>")]
    if len(input_widgets) > 0:
        inputs = input_widgets

    return VBox([
        widgets.HTML(value=f"<{h_size}>{title}</{h_size}>"),
        *inputs,
        add_btn,
    ], layout=inputs_layout)

def inputs_group_v2(h_size, title, grid, add_btn, is_empty=False):
    body = grid
    if is_empty:
        body = none

    return VBox([
        widgets.HTML(value=f"<{h_size}>{title}</{h_size}>"),
        body,
        add_btn
    ], layout=inputs_layout)


def add_input_button(text):
    return widgets.Button(description=text, layout=widgets.Layout(width='fit-content', margin='4px'))

def delete_income_button():
    return widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='auto', overflow='visible', margin='4px 8px'))

def field(label, widget):
    return VBox([Label(label), widget], layout=widgets.Layout(padding="0 4px"), justify_content='flex-start')

In [ ]:
class IncomeInput:
    def __init__(self, form, name: str="", amount: float=0.0, years: str="2024-2070", deposit_in=None, payroll_tax=True):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(value=name, continuous_update=False)
        self.amount_widget = widgets.FloatText(value=amount)
        self.years_widget = widgets.Text(value=years)
        self.account_dropdown = widgets.Dropdown(options=self.form.get_account_options(), value=deposit_in)
        self.federal_income_tax_widget = widgets.Checkbox(value=True)
        self.ny_income_tax_widget = widgets.Checkbox(value=True)
        self.nyc_income_tax_widget = widgets.Checkbox(value=True)
        self.payroll_tax_widget = widgets.Checkbox(value=payroll_tax)
        self.delete_btn = delete_income_button()

        # self.container = widgets.VBox([
        #     HBox([
        #         self.name_widget,
        #         self.amount_widget,
        #         self.years_widget,
        #         self.account_dropdown,
        #         self.federal_income_tax_widget,
        #         self.ny_income_tax_widget,
        #         self.nyc_income_tax_widget,
        #         self.payroll_tax_widget,
        #         self.delete_btn
        #         ], layout=widgets.Layout(width="100%")
        #     )
        # ], layout=input_layout)

        # Grid layout for this IncomeInput
        self.widgets_row = [
            self.name_widget,
            self.amount_widget,
            self.years_widget,
            self.account_dropdown,
            self.federal_income_tax_widget,
            self.ny_income_tax_widget,
            self.nyc_income_tax_widget,
            self.payroll_tax_widget,
            self.delete_btn
        ]

        self.delete_btn.on_click(self._on_delete)
        self.name_widget.observe(self._on_name_change, names='value')

    @classmethod
    def labels(cls):
        labels = ["Name", "Amount", "Years", "Deposit In", "Federal Income Tax", "NY Income Tax", "NYC Income Tax", "Payroll Tax", ""]
        return [widgets.Label(value=l, layout=widgets.Layout(overflow='visible', word_wrap='break-word', white_space='normal')) for l in labels]

    @classmethod
    def grid(cls, income_inputs):
      # Create a list to hold the grid elements
      income_grid_elements = []

      # Add the labels to the grid
      income_grid_elements.extend(cls.labels())

      # Add each IncomeInput to the grid
      for input_ in income_inputs:
          income_grid_elements.extend(input_.widgets_row)

      # Create the GridBox
      return widgets.GridBox(
          children=income_grid_elements,
          layout=widgets.Layout(
              width='100%',  # Limit the width of the entire grid to fit the screen
              grid_template_columns="repeat(9, minmax(100px, 1fr))",  # Adjust columns as needed
              grid_gap="10px 10px",  # Spacing between rows and columns
              overflow="auto"  # Allow horizontal scrolling if necessary
          )
      )

    def _on_delete(self, b):
        self.form.delete_income_input(self)

    def _on_name_change(self, change):
        self.form.update_account_dropdowns()

    def update_account_dropdown(self):
        self.account_dropdown.options = self.form.get_account_options()

    def get_data(self):
        years_list = parse_years(self.years_widget.value)
        return {
            "name": self.name_widget.value,
            "amount": self.amount_widget.value,
            "years": years_list,
            "deposit_in": self.account_dropdown.value,
            "federal_income_tax": self.federal_income_tax_widget.value,
            "payroll_tax": self.payroll_tax_widget.value,
            "ny_income_tax": self.ny_income_tax_widget.value,
            "nyc_income_tax": self.nyc_income_tax_widget.value,
        }


In [ ]:
class DebtInput:
    def __init__(self, form):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(description="Name:")
        self.amount_widget = widgets.FloatText(description="Amount", value=0.0)
        self.aagr_widget = widgets.FloatSlider(description="AAGR", min=0.0, max=1.0, step=0.001, value=0.0, continuous_update=True)
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.delete_btn.on_click(self._on_delete)

        self.container = HBox([
            self.name_widget,
            self.amount_widget,
            self.aagr_widget,
            self.delete_btn
        ], layout=input_layout)


    def _on_delete(self, b):
        self.form.delete_debt_input(self)

    def get_data(self):
        return {
            "name": self.name_widget.value,
            "amount": self.amount_widget.value,
            "aagr": self.aagr_widget.value
        }


In [ ]:
class InvestmentProportionInput:
    def __init__(self, form, parent_distribution_input):
        self.form = form
        self.parent_distribution_input = parent_distribution_input  # Reference to the parent InvestmentDistributionInput

        self.investment_widget = widgets.Dropdown(
            options=self.form.get_investment_vehicle_options(), description="Investment:")
        self.proportion_widget = widgets.FloatText(description="Proportion:", value=0.0)
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = HBox([
            self.investment_widget,
            self.proportion_widget,
            self.delete_btn,
        ])

        self.delete_btn.on_click(self._on_delete)

    def _on_delete(self, b):
        self.parent_distribution_input.delete_investment_proportion_input(self)

    def get_data(self):
        return {
            "investment_vehicle": self.investment_widget.value,
            "proportion": self.proportion_widget.value,
        }


In [ ]:
class InvestmentDistributionInput:
    def __init__(self, form, account_input):
        self.form = form
        self.account_input = account_input  # Reference to the parent AccountInput
        self.investment_proportion_inputs = []  # Store instances of InvestmentProportionInput

        self.years_widget = widgets.Text(description="Years:", continuous_update=False)
        self.investment_proportions_widget = widgets.VBox()
        self.add_investment_proportion_btn = add_input_button("Add Investment Proportion")
        self.delete_btn = widgets.Button(description="Delete Distribution", button_style='danger')

        self.add_investment_proportion_btn.on_click(self.add_investment_proportion_input)
        self.delete_btn.on_click(self._on_delete)

        self.container = widgets.VBox([
            self.years_widget,
            self.investment_proportions_widget,
            self.add_investment_proportion_btn,
            self.delete_btn
        ])

    def _on_delete(self, b):
        self.account_input.delete_investment_distribution_input(self)

    def add_investment_proportion_input(self, b=None):
        investment_proportion_input = InvestmentProportionInput(self.form, self)
        self.investment_proportion_inputs.append(investment_proportion_input)
        self.investment_proportions_widget.children = list(self.investment_proportions_widget.children) + [investment_proportion_input.container]

    def delete_investment_proportion_input(self, investment_proportion_input):
        if investment_proportion_input in self.investment_proportion_inputs:
            self.investment_proportion_inputs.remove(investment_proportion_input)
            self.investment_proportions_widget.children = [input.container for input in self.investment_proportion_inputs]

    def get_data(self):
        years = parse_years(self.years_widget.value)
        investment_proportions = [input.get_data() for input in self.investment_proportion_inputs]
        return {
            "years": years,
            "investment_proportions": investment_proportions,
        }


In [ ]:
class InvestmentVehicleInput:
    def __init__(self, form, name="", aagr=0.0, dynamic_mean=0.0, dynamic_std_dev=0.0):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(description="Name:", value=name, continuous_update=False)
        self.aagr_widget = widgets.FloatText(description="AAGR:", value=aagr)
        self.dynamic_mean_widget = widgets.FloatText(description="Dynamic Mean:", value=dynamic_mean)
        self.dynamic_std_dev_widget = widgets.FloatText(description="Dynamic Std Dev:", value=dynamic_std_dev)
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = HBox([
            self.name_widget,
            self.aagr_widget,
            self.dynamic_mean_widget,
            self.dynamic_std_dev_widget,
            self.delete_btn
        ])

        self.delete_btn.on_click(self._on_delete)
        self.name_widget.observe(self._on_name_change, names='value')

    def _on_delete(self, b):
        self.form.delete_investment_vehicle_input(self)
        self.form.update_investment_vehicle_dropdowns()

    def _on_name_change(self, change):
        self.form.update_investment_vehicle_dropdowns()

    def get_data(self):
        return {
            "name": self.name_widget.value,
            "aagr": self.aagr_widget.value,
            "dynamic_mean": self.dynamic_mean_widget.value,
            "dynamic_std_dev": self.dynamic_std_dev_widget.value,
        }


In [ ]:
class AccountInput:
    def __init__(self, form, name="", account_type=ACCOUNT_TYPE.BANK, starting_balance=0.0, earliest_withdrawal_year=2024):
        self.form = form  # Reference to the form instance
        self.investment_distribution_inputs = []  # Store instances of InvestmentDistributionInput


        self.name_widget = widgets.Text(description="Name:", value=name, continuous_update=False)
        self.account_type_widget = widgets.Dropdown(options=ACCOUNT_TYPE.ALL, description="Type:", value=account_type)
        self.starting_balance_widget = widgets.FloatText(description="Balance", value=starting_balance)
        self.earliest_withdrawal_year_widget = widgets.IntSlider(min=2024, max=2070, step=1, value=earliest_withdrawal_year, continuous_update=True)

        self.investment_distributions_widget = widgets.VBox(layout=widgets.Layout(border='solid 0.5px gray', padding='10px', margin='10px', width='auto'))
        self.add_investment_distribution_btn = add_input_button("Add Investment Distribution")
        self.add_investment_distribution_btn.on_click(self.add_investment_distribution_input)

        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = VBox([
            HBox([
              self.name_widget,
              self.account_type_widget,
              self.starting_balance_widget,
              VBox([widgets.Label("Earliest Withdrawal Year"), self.earliest_withdrawal_year_widget]),
              self.delete_btn
            ]),
            widgets.HTML(value="<h3>Investment Distributions</h3>"),
            self.investment_distributions_widget,
            self.add_investment_distribution_btn
        ], layout=input_layout)

        self.delete_btn.on_click(self._on_delete)
        self.name_widget.observe(self._on_name_change, names='value')

    def add_investment_distribution_input(self, b=None):
        investment_distribution_input = InvestmentDistributionInput(self.form, self)
        self.investment_distribution_inputs.append(investment_distribution_input)
        self.investment_distributions_widget.children = list(self.investment_distributions_widget.children) + [investment_distribution_input.container]

    def delete_investment_distribution_input(self, investment_distribution_input):
        if investment_distribution_input in self.investment_distribution_inputs:
            self.investment_distribution_inputs.remove(investment_distribution_input)
            # Update the widget display after deletion
            self.investment_distributions_widget.children = [input.container for input in self.investment_distribution_inputs]

    def update_investment_vehicle_dropdowns(self):
        for investment_distribution_input in self.investment_distribution_inputs:
            for investment_proportion_input in investment_distribution_input.investment_proportions_widget.children:
                investment_proportion_input.investment_widget.options = self.form.get_investment_vehicle_options()

    def _on_delete(self, b):
        self.form.delete_account_input(self)

    def _on_name_change(self, change):
        self.form.update_account_dropdowns()

    def get_data(self):
        return {
            "name": self.name_widget.value,
            "account_type": self.account_type_widget.value,
            "starting_balance": self.starting_balance_widget.value,
            "earliest_withdrawal_year": self.earliest_withdrawal_year_widget.value,
            "investment_distributions": [input.get_data() for input in self.investment_distribution_inputs]
        }


In [ ]:
class ExpenseInput:
    def __init__(self, form):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(description="Name:", continuous_update=False)
        self.amount_widget = widgets.FloatText(description="Amount", value=0.0)
        self.years_widget = widgets.Text(description="Years:", value="")
        self.five_two_nine_eligible_widget = widgets.Checkbox(description="529 Eligible", value=False)
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = HBox([
            self.name_widget,
            self.amount_widget,
            self.years_widget,
            self.five_two_nine_eligible_widget,
            self.delete_btn
        ])

        self.delete_btn.on_click(self._on_delete)

    def _on_delete(self, b):
        self.form.delete_expense_input(self)

    def get_data(self):
        years_list = parse_years(self.years_widget.value)
        return {
            "name": self.name_widget.value,
            "amount": self.amount_widget.value,
            "years": years_list,
            "five_two_nine_eligible": self.five_two_nine_eligible_widget.value,
        }


In [ ]:
class GiftInput:
    def __init__(self, form):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(description="Name:", continuous_update=False)
        self.amount_widget = widgets.FloatText(description="Amount", value=0.0)
        self.years_widget = widgets.Text(description="Years:", value="")
        self.account_dropdown = widgets.Dropdown(options=self.form.get_account_options(), description="Account:")
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = HBox([
            self.name_widget,
            self.amount_widget,
            self.years_widget,
            self.account_dropdown,
            self.delete_btn
        ])

        self.delete_btn.on_click(self._on_delete)

    def _on_delete(self, b):
        self.form.delete_gift_input(self)

    def update_account_dropdown(self):
        self.account_dropdown.options = self.form.get_account_options()

    def get_data(self):
        years_list = parse_years(self.years_widget.value)
        return {
            "name": self.name_widget.value,
            "amount": self.amount_widget.value,
            "years": years_list,
            "account": self.account_dropdown.value,
        }


In [ ]:
class TransferInput:
    def __init__(self, form):
        self.form = form  # Reference to the form instance

        self.name_widget = widgets.Text(description="Name:", continuous_update=False)
        self.amount_widget = widgets.FloatText(description="Amount", value=0.0)
        self.years_widget = widgets.Text(description="Years:", value="")
        self.transfer_from_widget = widgets.Dropdown(options=self.form.get_account_options(), description="From:")
        self.transfer_to_widget = widgets.Dropdown(options=self.form.get_account_options(), description="To:")
        self.required_widget = widgets.Checkbox(description="Required", value=False)
        self.delete_btn = widgets.Button(description="Delete", button_style='danger')

        self.container = HBox([
            self.name_widget,
            self.amount_widget,
            self.years_widget,
            self.transfer_from_widget,
            self.transfer_to_widget,
            self.required_widget,
            self.delete_btn
        ])

        self.delete_btn.on_click(self._on_delete)

    def _on_delete(self, b):
        self.form.delete_transfer_input(self)

    def update_account_dropdown(self):
        self.transfer_from_widget.options = self.form.get_account_options()
        self.transfer_to_widget.options = self.form.get_account_options()

    def get_data(self):
        years_list = parse_years(self.years_widget.value)
        return {
            "name": self.name_widget.value,
            "amount": self.amount_widget.value,
            "years": years_list,
            "transfer_from": self.transfer_from_widget.value,
            "transfer_to": self.transfer_to_widget.value,
            "required": self.required_widget.value,
        }


In [ ]:
class Form:
    def __init__(self):
        self.investment_vehicle_inputs = []  # List to store instances of InvestmentVehicle
        self.debt_inputs = []  # List to store instances of DebtInput
        self.account_inputs = []  # List to store instances of AccountInput
        self.income_inputs = []  # List to store instances of IncomeInput
        self.expense_inputs = []  # List to store instances of ExpenseInput
        self.gift_inputs = []  # List to store instances of GiftInput
        self.transfer_inputs = []  # List to store instances of TransferInput
        self.display_area = widgets.Output()  # Output widget to manage in-place updates

        # Create the submit button once
        self.submit_btn = widgets.Button(description="Submit", button_style="info")
        self.submit_btn.on_click(self.handle_submit)

        self.results_displayer = ResultsDisplayer()


    #####  DEBTS  #####
    def add_debt_input(self, b=None):
        debt_input = DebtInput(self)
        self.debt_inputs.append(debt_input)
        self.update_display()

    def delete_debt_input(self, debt_input):
        if debt_input in self.debt_inputs:
            self.debt_inputs.remove(debt_input)
        self.update_display()

    #####  ACCOUNTS  #####
    def add_account_input(self, b=None, name="", account_type=ACCOUNT_TYPE.BANK, starting_balance=0.0, earliest_withdrawal_year=2024):
        account_input = AccountInput(self, name, account_type, starting_balance, earliest_withdrawal_year)
        self.account_inputs.append(account_input)
        self.update_display()

    def delete_account_input(self, account_input):
        if account_input in self.account_inputs:
            self.account_inputs.remove(account_input)
            self.update_account_dropdowns()
        self.update_display()

    def update_account_dropdowns(self):
        for gift_input in self.gift_inputs:
            gift_input.update_account_dropdown()
        for transfer_input in self.transfer_inputs:
            transfer_input.update_account_dropdown()
        for income_input in self.income_inputs:
            income_input.update_account_dropdown()

    def get_account_options(self):
        all_data = self.get_all_data()
        return [account["name"] for account in all_data["accounts"]]

    #####  INCOMES  #####
    def add_income_input(self, b=None, name="", amount=0.0, deposit_in=None, years="", payroll_tax=True):
        income_input = IncomeInput(self, name=name, amount=amount, deposit_in=deposit_in, years=years, payroll_tax=payroll_tax)
        self.income_inputs.append(income_input)
        self.update_display()

    def delete_income_input(self, income_input):
        if income_input in self.income_inputs:
            self.income_inputs.remove(income_input)
        self.update_display()



    #####  EXPENSES  #####
    def add_expense_input(self, b=None):
        expense_input = ExpenseInput(self)
        self.expense_inputs.append(expense_input)
        self.update_display()

    def delete_expense_input(self, expense_input):
        if expense_input in self.expense_inputs:
            self.expense_inputs.remove(expense_input)
        self.update_display()

    #####  GIFTS  #####
    def add_gift_input(self, b=None):
        gift_input = GiftInput(self)
        self.gift_inputs.append(gift_input)
        self.update_display()

    def delete_gift_input(self, gift_input):
        if gift_input in self.gift_inputs:
            self.gift_inputs.remove(gift_input)
        self.update_display()

    #####  TRANSFERS  #####
    def add_transfer_input(self, b=None):
        transfer_input = TransferInput(self)
        self.transfer_inputs.append(transfer_input)
        self.update_display()

    def delete_transfer_input(self, transfer_input):
        if transfer_input in self.transfer_inputs:
            self.transfer_inputs.remove(transfer_input)
        self.update_display()

    #####  INVESTMENT VEHICLES  #####
    def add_investment_vehicle_input(self, b=None, name="", aagr=0.0, dynamic_mean=0.0, dynamic_std_dev=0.0):
        investment_vehicle_input = InvestmentVehicleInput(self, name, aagr, dynamic_mean, dynamic_std_dev)
        self.investment_vehicle_inputs.append(investment_vehicle_input)
        self.update_display()

    def delete_investment_vehicle_input(self, investment_vehicle_input):
        if investment_vehicle_input in self.investment_vehicle_inputs:
            self.investment_vehicle_inputs.remove(investment_vehicle_input)
        self.update_display()
        self.update_investment_vehicle_dropdowns()

    def get_investment_vehicle_options(self):
        return [input.get_data()["name"] for input in self.investment_vehicle_inputs]

    def update_investment_vehicle_dropdowns(self):
        # Loop through each account and update the dropdowns
        for account_input in self.account_inputs:
            for investment_distribution_input in account_input.investment_distribution_inputs:
                for investment_proportion_input in investment_distribution_input.investment_proportion_inputs:
                    investment_proportion_input.investment_widget.options = self.get_investment_vehicle_options()


    #####  DISPLAY  #####
    def update_display(self):
        with self.display_area:
            clear_output(wait=True)

            investment_vehicle_widgets = [input.container for input in self.investment_vehicle_inputs]
            account_widgets = [account_input.container for account_input in self.account_inputs]
            debt_widgets = [debt_input.container for debt_input in self.debt_inputs]
            expense_widgets = [expense_input.container for expense_input in self.expense_inputs]
            gift_widgets = [gift_input.container for gift_input in self.gift_inputs]
            transfer_widgets = [transfer_input.container for transfer_input in self.transfer_inputs]


            add_investment_vehicle_btn = add_input_button("Add Investment Vehicle")
            add_investment_vehicle_btn.on_click(self.add_investment_vehicle_input)

            add_account_btn = widgets.Button(description="Add Account")
            add_account_btn.on_click(self.add_account_input)

            add_debt_btn = widgets.Button(description="Add Debt")
            add_debt_btn.on_click(self.add_debt_input)

            add_income_btn = widgets.Button(description="Add Income")
            add_income_btn.on_click(self.add_income_input)

            add_expense_btn = widgets.Button(description="Add Expense")
            add_expense_btn.on_click(self.add_expense_input)

            add_gift_btn = widgets.Button(description="Add Gift")
            add_gift_btn.on_click(self.add_gift_input)

            add_transfer_btn = widgets.Button(description="Add Transfer")
            add_transfer_btn.on_click(self.add_transfer_input)

            display(widgets.VBox([
                inputsGroup("h2", "Investment Vehicles", investment_vehicle_widgets, add_investment_vehicle_btn),
                inputsGroup("h2", "Accounts", account_widgets, add_account_btn),
                inputsGroup("h2", "Debt", debt_widgets, add_debt_btn),
                inputs_group_v2("h2", "Income", IncomeInput.grid(self.income_inputs), add_income_btn, is_empty=(len(self.income_inputs) == 0)),
                inputsGroup("h2", "Expenses", expense_widgets, add_expense_btn),
                inputsGroup("h2", "Gifts", gift_widgets, add_gift_btn),
                inputsGroup("h2", "Transfers", transfer_widgets, add_transfer_btn),
                self.submit_btn
            ]))

    def display(self):
        self.add_investment_vehicle_input(self, name="s&p", aagr=0.0655, dynamic_mean=0.077, dynamic_std_dev=0.175)
        self.add_investment_vehicle_input(self, name="bonds", aagr=0.0352, dynamic_mean=0.039, dynamic_std_dev=0.0855)
        self.add_account_input(self, name="bank of america", account_type="bank", starting_balance=30000.0)
        self.add_account_input(self, name="fidelity", account_type="retirement", starting_balance=500000.0, earliest_withdrawal_year=2039)
        self.add_income_input(self, name="spouse 1 income", amount="100000.0", deposit_in="bank of america", years="2024-2048")
        self.add_income_input(self, name="spouse 2 income", amount="100000.0", deposit_in="bank of america", years="2024-2048")
        self.add_income_input(self, name="spouse 1 social security", amount="35000.0", deposit_in="bank of america", years="2049-2070", payroll_tax=False)
        self.add_income_input(self, name="spouse 2 social security", amount="35000.0", deposit_in="bank of america", years="2049-2070", payroll_tax=False)
        display(self.display_area)
        self.update_display()

    def get_all_data(self):
        return {
            "debts": [debt_input.get_data() for debt_input in self.debt_inputs],
            "accounts": [account_input.get_data() for account_input in self.account_inputs],
            "incomes": [income_input.get_data() for income_input in self.income_inputs],
            "expenses": [expense_input.get_data() for expense_input in self.expense_inputs],
            "gifts": [gift_input.get_data() for gift_input in self.gift_inputs],
            "transfers": [transfer_input.get_data() for transfer_input in self.transfer_inputs],
            "investment_vehicles": [investment_vehicle_input.get_data() for investment_vehicle_input in self.investment_vehicle_inputs]
        }

    def handle_submit(self, b):
        data = self.get_all_data()
        first_year = 2024
        last_year = 2070
        dynamic = False
        debug = False
        results_data = Simulations(data, first_year=first_year, last_year=last_year, dynamic=dynamic, debug=dynamic).execute()
        self.results_displayer.display(first_year=first_year, last_year=last_year, aggregator=results_data, dynamic=dynamic, debug=debug)


In [ ]:
form = Form()
form.display()

In [ ]:
# first_year = 2024
# last_year = 2070
# dynamic = True
# debug = False

# data = {'debts': [{'name': 'some random debt', 'amount': 1000.0, 'aagr': 0.03}], 'accounts': [{'name': 'vanguard', 'account_type': 'investments', 'starting_balance': 150000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': [{'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}]}, {'name': 'fidelity', 'account_type': 'retirement', 'starting_balance': 730000.0, 'earliest_withdrawal_year': 2039, 'investment_distributions': [{'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2031, 2032, 2033, 2034, 2035, 2036], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.8}, {'investment_vehicle': 'bonds', 'proportion': 0.2}]}, {'years': [2037, 2038, 2039, 2040, 2041, 2042], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.7}, {'investment_vehicle': 'bonds', 'proportion': 0.3}]}, {'years': [2043, 2044, 2045, 2046, 2047, 2048], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.6}, {'investment_vehicle': 'bonds', 'proportion': 0.4}]}, {'years': [2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.5}, {'investment_vehicle': 'bonds', 'proportion': 0.5}]}]}, {'name': 'boa', 'account_type': 'bank', 'starting_balance': 30000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': []}, {'name': 'nysaves', 'account_type': '529', 'starting_balance': 85000.0, 'earliest_withdrawal_year': 2024, 'investment_distributions': [{'years': [2024, 2025, 2026], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.9}, {'investment_vehicle': 'bonds', 'proportion': 0.1}]}, {'years': [2027, 2028, 2029], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.8}, {'investment_vehicle': 'bonds', 'proportion': 0.2}]}, {'years': [2030, 2031, 2032], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.7}, {'investment_vehicle': 'bonds', 'proportion': 0.3}]}, {'years': [2033, 2034, 2035, 2036], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.6}, {'investment_vehicle': 'bonds', 'proportion': 0.4}]}, {'years': [2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'investment_proportions': [{'investment_vehicle': 'stocks', 'proportion': 0.5}, {'investment_vehicle': 'bonds', 'proportion': 0.5}]}]}], 'incomes': [{'name': 'david full-time', 'amount': 130000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'claire full-time', 'amount': 120000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'david part-time', 'amount': 70000.0, 'years': [2040, 2041, 2042, 2043, 2044], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}, {'name': 'claire part-time', 'amount': 80000.0, 'years': [2040, 2041, 2042, 2043, 2044], 'deposit_in': 'boa', 'federal_income_tax': True, 'payroll_tax': True, 'ny_income_tax': True, 'nyc_income_tax': True}], 'expenses': [{'name': 'car', 'amount': 40000.0, 'years': [2027, 2037, 2047, 2057, 2067], 'five_two_nine_eligible': False}, {'name': 'zach college', 'amount': 50000.0, 'years': [2039, 2040, 2041, 2042], 'five_two_nine_eligible': True}, {'name': 'sloane college', 'amount': 50000.0, 'years': [2037, 2038, 2039, 2040], 'five_two_nine_eligible': True}, {'name': 'copake', 'amount': 12000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044], 'five_two_nine_eligible': False}, {'name': 'every day expenses', 'amount': 101500.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070], 'five_two_nine_eligible': False}, {'name': 'zach daycare', 'amount': 20000.0, 'years': [2024, 2025, 2026], 'five_two_nine_eligible': False}], 'gifts': [{'name': 'dad 529', 'amount': 8000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037], 'account': 'nysaves'}], 'transfers': [{'name': 'retirement contribution', 'amount': 45000.0, 'years': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039], 'transfer_from': 'boa', 'transfer_to': 'fidelity', 'required': False}], 'investment_vehicles': [{'name': 'stocks', 'aagr': 0.0655, 'dynamic_mean': 0.077, 'dynamic_std_dev': 0.1175}, {'name': 'bonds', 'aagr': 0.0352, 'dynamic_mean': 0.039, 'dynamic_std_dev': 0.0855}]}
# results = Simulations(data, first_year=first_year, last_year=last_year, dynamic=dynamic, debug=dynamic).execute()
# ResultsDisplay(first_year=first_year, last_year=last_year, aggregator=results, dynamic=dynamic, debug=debug).display()